# Advanced Lane Finding Project

---

**Author:** Sergey Morozov

---

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [ ]:
# set up logging
import logging
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s [%(levelname)s] %(message)s")

# constants
CHESSBOARD_IMAGE_DIR = "chessboard_images"
TEST_IMAGE_DIR = "test_images"
OUTPUT_IMAGE_DIR = "output_images"
OUTPUT_VIDEO_DIR = "output_videos"

# all lane finding code is in advanced_lane_finding.py
from advanced_lane_finding import *

# initialize class instance containing advanced lane line detection methods
lane_finder = AdvancedLaneFinder(chessboard_image_dir=CHESSBOARD_IMAGE_DIR)

## Compute Camera Calibration Matrix and Distortion Coefficients

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# get calibration matrix and distortion coefficients
cal_mtx, dist_coeffs, _, _ = lane_finder.calibrate_camera()

# undistort chessboard image calibration2.jpg;
# visially it is the mostly distorted image
test_img = cv2.imread(os.path.join(CHESSBOARD_IMAGE_DIR, "calibration2.jpg"))

# undistort test chessboard image
test_img_undistorted = lane_finder.distortion_correction(image=test_img)

# visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(test_img_undistorted)
ax2.set_title('Undistorted Image', fontsize=30)

# save undistorted image
if not os.path.isdir(OUTPUT_IMAGE_DIR):
    os.mkdir(OUTPUT_IMAGE_DIR)

undist_img_path = os.path.abspath(os.path.join(OUTPUT_IMAGE_DIR, "chessboard_undistorted.jpg"))
ret = cv2.imwrite(undist_img_path, test_img_undistorted)
logging.info("Undistorted chessboard image has been written to %s.", undist_img_path)

Note that in the initial set of chessboard images, provided by Udacity, image *calibration7.jpg* and *calibration15.jpg* had shape (1281, 721) while all other images had shape (1280, 720). These discrepancy led to inaccrate calculations. Images with incorrect shape (*calibration7.jpg* and *calibration15.jpg*) were cropped by 1 pixel and in this repository all images have (1280, 720) shape.

## Apply Distortion Correction to Raw Images

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
%matplotlib inline

# undistort each image in TEST_IMAGE_DIR and save undistorted images to OUTPUT_IMAGE_DIR
test_img_paths = os.listdir(TEST_IMAGE_DIR)

for fname in test_img_paths:
    img = cv2.imread(os.path.join(TEST_IMAGE_DIR, fname))
    img_undist = lane_finder.distortion_correction(image=img)
    
    # save undistorted image
    cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, fname[0:-4] + "_undistorted.jpg"), img_undist)
    
# visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))

img = plt.imread(os.path.join(TEST_IMAGE_DIR, "test1.jpg"))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)

img_undist = plt.imread(os.path.join(OUTPUT_IMAGE_DIR, "test1_undistorted.jpg"))
ax2.imshow(img_undist)
ax2.set_title('Undistorted Image', fontsize=30)

## Create Thresholded Binary Images

In [ ]:
import cv2 
import os
import matplotlib.pyplot as plt
%matplotlib inline

# apply thretholds for each image in TEST_IMAGE_DIR and save thresholded images to OUTPUT_IMAGE_DIR
test_img_paths = os.listdir(TEST_IMAGE_DIR)

for fname in test_img_paths:
    img = cv2.imread(os.path.join(TEST_IMAGE_DIR, fname))
    img_thresh = lane_finder.apply_thresholds(image=img)
    
    # save undistorted image
    cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, fname[0:-4] + "_thresholded.jpg"), img_thresh)

# visualize thresholded images (use difficult cases)
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20,10))

img = plt.imread(os.path.join(TEST_IMAGE_DIR, "test4.jpg"))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)

img_thresh = plt.imread(os.path.join(OUTPUT_IMAGE_DIR, "test4_thresholded.jpg"))
ax2.imshow(img_thresh, cmap='gray')
ax2.set_title('Thresholded Image', fontsize=30)

img = plt.imread(os.path.join(TEST_IMAGE_DIR, "test5.jpg"))
ax3.imshow(img)
ax3.set_title('Original Image', fontsize=30)

img_thresh = plt.imread(os.path.join(OUTPUT_IMAGE_DIR, "test5_thresholded.jpg"))
ax4.imshow(img_thresh, cmap='gray')
ax4.set_title('Thresholded Image', fontsize=30)